# Exercise 2 - Interactive charting with plotly

## 1. Internet usage historical

In [1]:
#   a) Recreate this visualization using plotly, make it as close as possible to the image.
import pandas as pd

df_internet = pd.read_csv("data/share-of-individuals-using-the-internet.csv")

In [2]:
import duckdb

df_selection = duckdb.query("""--sql
    SELECT 
        Entity,
        Year,
        "Individuals using the Internet (% of population)"
    FROM df_internet
    WHERE Entity IN (
                    'East Asia and Pacific (WB)',
                    'Europe and Central Asia (WB)', 
                    'Latin America and Caribbean (WB)',
                    'Middle East and North Africa (WB)',
                    'North America (WB)',
                    'South Asia (WB)',
                    'Sub-Saharan Africa (WB)',
                    'World'  )
""").to_df()

df_selection

,Entity,Year,Individuals using the Internet (% of population)
0,Europe and Central Asia (WB),1990,0.048031
1,Europe and Central Asia (WB),1991,0.101868
2,Europe and Central Asia (WB),1992,0.180602
3,Europe and Central Asia (WB),1993,0.279651
4,Europe and Central Asia (WB),1994,0.524235
...,...,...,...
265,World,2019,53.200000
266,World,2020,59.300000
267,World,2021,62.200000
268,World,2022,64.400000


In [ ]:
import plotly.express as px

top_title = "#2D2E2D"
sub_title = "#5B5B5B"
y_label_color = "#555555"
explanations_color = "#858585"

color_dict = {
    "North America (WB)": "#ab8fbf",
    "Europe and Central Asia (WB)": "#cb7b5d",
    "Latin America and Caribbean (WB)": "#c1a788",
    "East Asia and Pacific (WB)": "#a1b0ca" ,
    "Middle East and North Africa (WB)": "#6eaa95",
    "World": "#c99dc5",
    "South Asia (WB)": "#b0777d",
    "Sub-Saharan Africa (WB)": "#405e84"
}


# ----------------------------------------------------------#

fig = px.line(
    df_selection,
    x="Year",
    y="Individuals using the Internet (% of population)",
    color="Entity",  # This lets you draw a separate line per region or group
    color_discrete_map=color_dict,
    markers=True
)

fig.update_layout(
    width=1000,
    height=800,
    margin=dict(r=120, l=40,b=120,t=80),  # give space for text
    plot_bgcolor="white",
    paper_bgcolor="white",
    showlegend=False
)


# Get default tick values based on your data range
tickvals = list(range(0, 101, 10))  # Adjust range/step as needed
ticktext = [f"{v}%" for v in tickvals]

# Update layout with title
fig.update_layout(
    title=dict(
        text=(
            "<span style='font-size:28px; color:top_title;font-weight:bold;'>"
            "Share of the population using the Internet"
            "</span><br>"
            "<span style='font-size:17px; color:sub_title;'>"
            "Share of the population who used the Internet<sup>1</sup> in the last three months."
            "</span>"
        ),
        x=0.01,  # Position title
        y=0.95,
        xanchor="left",
    ),
    xaxis_title=None,
    yaxis_title=None,
    yaxis=dict(
        tickvals=tickvals,
        ticktext=ticktext,
        tickfont=dict(size=16, color=y_label_color),
        showgrid=True,
        gridcolor="#E5E5E5",  # optional: light gray grid lines
        gridwidth=1,
        griddash="dash",
        showline=True,
        showticklabels=True,
        domain=[0.0, 0.99],
    ),
    xaxis=dict(
        range=[1990, 2023.2],  # Fix the x-axis to desired bounds
        showgrid=False,
        showline=True,
        showticklabels=False, 
        domain=[0, 0.8],   
        linecolor="#ccc"
    ),
)

fig.update_xaxes(
    anchor="free",
    position = 0.055,
    tickvals=[1990, 1995, 2000, 2005, 2010, 2015, 2023],  # or auto
    ticktext=["1990", "1995", "2000", "2005", "2010",  "2015","2023"],
    tickfont=dict(size=14, color="#555555"),
    ticks="outside",
    showticklabels=True)

# Annotate each line with its label
# List of entities in desired vertical order (top to bottom)
entities_ordered = list(color_dict.keys())  # You can reorder manually if you like

# Custom vertical positions (same length as entities_ordered)
custom_y_positions = [
    0.96,  # North America
    0.89,  # Europe and Central Asia
    0.82,  # Latin America and Caribbean
    0.79,  # East Asia and Pacific
    0.76,  # Middle East and North Africa
    0.68,  # World
    0.46,  # South Asia
    0.4   # Sub-Saharan Africa
]

for i, entity in enumerate(entities_ordered):
    y_pos = custom_y_positions[i]

    # Draw the annotation
    fig.add_annotation(
        x=0.817,
        y=y_pos,
        xref="paper",
        yref="paper",
        text=entity,
        font=dict(color=color_dict.get(entity, "#000000"), size=14),
        showarrow=False,
        xanchor="left"
    )

    # Draw a short gray line from near the right edge of the plot to the label
    fig.add_shape(
        type="line",
        xref="paper",
        yref="paper",
        x0=0.795,  # end of data area (~0.8), adjust if needed
        x1=0.812,  # just before the label
        y0= y_pos,
        y1= y_pos,
        line=dict(color="#545252", width=1),
        layer="above"
    )

# Update layout with bottom explanation text
fig.add_annotation(
    text=(
        "<span style='font-size:14px; color:" + explanations_color + ";'>"
        "<b>Data source:</b> International Telecommunication Union (via World Bank) (2025)"
        "                              OurWorldinData.org/internet | CC BY"
        "</span>"
    ),
    xref="paper",  # Relative to the plot width (0 to 1)
    yref="paper",  # Relative to the plot height (0 to 1)
    x=-0.06,  # Match title x position (left-aligned)
    y=-0.04,  # Below the plot (adjust as needed)
    showarrow=False,
    align="left",
)

# Separation line
fig.add_shape(
    type="line",
    xref="paper",
    yref="paper",
    x0= -0.06,
    x1= 1.12,
    y0= -0.06,
    y1= -0.06,
    line=dict(color=explanations_color, width=1)
)

# Additional explanatory text
fig.add_annotation(
    text="<span style='font-size:13px; color:" + explanations_color + ";'>"
         "<b>1. Internet user:</b>An internet user is defined by the International Telecommunication Union as anyone who has accessed the internet from any<br>"
         "location in the last three months. This can be from any type of device, including a computer, mobile phone, personal digital asssistant, games<br>"
         "machine, digital TV, and other technological devices.</span>",
    xref="paper",
    yref="paper",
    x=-0.06,
    y=-0.16,
    showarrow=False,
    align="left"
)
fig.add_layout_image(
    dict(
        source="figures/logo.png",  
        xref="paper", yref="paper",  # Relative to figure, not data
        x=1.1, y=1.04,                  # Bottom-right corner
        sizex=0.08, sizey=0.08,      # Adjust size as needed
        xanchor="right", yanchor="bottom",
        layer="above"               # Draw above plot
    )
)
fig.show()